In [ ]:
# 필요 라이브러리 import 
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
import warnings
warnings.filterwarnings(action='ignore')

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

from sklearn.datasets import make_classification
from collections import Counter

from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
train = pd.read_csv("../data/8-1 train(std).csv",index_col=0)
test1 = pd.read_csv("../data/8-1 test1(std).csv",index_col=0)
test2 = pd.read_csv("../data/8-1 test2(std).csv",index_col=0)

print(train['label'].value_counts())
print(test1['label'].value_counts())
print(test2['label'].value_counts())

데이터분할

In [ ]:
x_train = train.drop("label",axis=1)
y_train = train["label"]
x_test =test2.drop("label",axis=1)
y_test = test2["label"]

오버샘플링 SMOTE

In [ ]:
x_train, y_train = SMOTE(random_state=4).fit_resample(x_train,y_train)
print(x_train.shape)
y_train.value_counts()

modeling

In [ ]:
# 성능확인 코드
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score


import warnings

def get_clf_eval(MDA_y_test, pred):
    confusion = confusion_matrix(MDA_y_test, pred)
    accuracy = accuracy_score(MDA_y_test, pred)
    precision = precision_score(MDA_y_test, pred)
    recall = recall_score(MDA_y_test, pred)
    roc_score = roc_auc_score(MDA_y_test, pred)
    pr_score = average_precision_score(MDA_y_test, pred)
    f1 = f1_score(MDA_y_test, pred)
    print('오차행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도 : {1:.4f}, 재현율:{2:.4f},F1 스코어:{3:.4f}'.format(accuracy, precision, recall, f1, roc_score))
    print('ROC 스코어: {0:.4f}, PR score : {1:.4f}'.format(roc_score, pr_score))

In [ ]:
# 임계값에 따른 오차행렬및 스코어 -------->#임계값 최적 : 재현율기준(0.1)/f1기준(0.3)
from sklearn.preprocessing import Binarizer
thresholds = [0.1,0.2,0.3,0.4,0.5]

def get_eval_by_threshold(MDA_y_test, pred_proba_c1, thresholds):
    for custom_threshold in thresholds:
        binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_c1)
        custom_predict = binarizer.transform(pred_proba_c1)
        print("임곗값:", custom_threshold)
        get_clf_eval(MDA_y_test, custom_predict)
        print("---------------------------------------------------------")



In [ ]:
# Roc Auc 그래프 값 확인 코드
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
def roc_curve_plot(MDA_y_test, pred_proba_c1):
    fprs, tprs, thresholds = roc_curve(MDA_y_test, pred_proba_c1)
    
    plt.plot(fprs, tprs, label='ROC')
    plt.plot([0,1],[0,1], 'k--', label = 'Random')
    
    start, end= plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1),2))
    plt.xlim(0,1); plt.ylim(0,1)
    plt.xlabel('FPR(1-Sensitivity)'); plt.ylabel('TPR(Recall)')
    plt.legend()
    plt.show()

LDA

In [ ]:
# 1-1 LDA 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda_clf = LinearDiscriminantAnalysis()

lda_clf.fit(x_train, y_train)
lda_pred = lda_clf.predict(x_test)
lda_pred_proba = lda_clf.predict_proba(x_test)
print("---------------------------------------------------------") 
get_clf_eval(y_test, lda_pred)
get_eval_by_threshold(y_test , lda_pred_proba[:,1].reshape(-1,1), thresholds)
    

Dicisiontree

In [ ]:
# 1-2 디시전트리 
from sklearn.tree import DecisionTreeClassifier 
dt_clf = DecisionTreeClassifier(random_state=52)

dt_clf.fit(x_train, y_train)
dt_pred = dt_clf.predict(x_test)
dt_pred_proba = dt_clf.predict_proba(x_test)   

get_clf_eval(y_test, dt_pred)
get_eval_by_threshold(y_test , dt_pred_proba[:,1].reshape(-1,1), thresholds)
    


In [ ]:
from sklearn.tree import DecisionTreeClassifier 
dt_clf = DecisionTreeClassifier(random_state=52)
from sklearn.model_selection import GridSearchCV
tree = DecisionTreeClassifier(random_state=52)

# key: 하이퍼파라미터 명; value: [후보값들] 을 리스트로 전달
param_grid = {
    'criterion': ['entropy', 'gini'],
            'max_depth': [2,4,6,8],
            'max_features': ['auto', 'sqrt'],
            'min_samples_leaf': [4,6,8,10],
            'min_samples_split': [4,6,8,10]}
grid_search = GridSearchCV(tree
                        , param_grid = param_grid
                        , cv = 5 #교차검증시 fold 개수; n번의 교차검증; 데이터들이 돌아가면서 하나씩 검증(validation)에 사용되어 일반화하는 것; 데이터가 많지 않을 때 보통 쓰인다고 함
                        , n_jobs = -1)  #-1: 모든 코어 사용

grid_search.fit(x_train, y_train)

# 최선의 결과 출력
print('최적 하이퍼 파라미터 :{0}, 최적 평균 정확도 :{1:.3f}'.format(grid_search.best_params_, grid_search.best_score_))
a = grid_search.best_params_
keys=list(a.keys())
values = list(a.values())

dt_clf_best = DecisionTreeClassifier(random_state=52,criterion = values[0], max_depth = values[1],max_features=values[2], min_samples_leaf=values[3], min_samples_split=values[4])
dt_clf_best.fit(x_train, y_train)
dt_best_pred = dt_clf_best.predict(x_test)
dt_best_pred_proba = dt_clf_best.predict_proba(x_test)   

print("디시전트리 파라미터적용") 
get_clf_eval(y_test, dt_best_pred)
get_eval_by_threshold(y_test ,dt_best_pred_proba[:,1].reshape(-1,1), thresholds)


In [ ]:
print("디시전트리") 
print(get_clf_eval(y_test, lda_pred))

RF

In [ ]:
# 1-3 랜덤포레스트 구현 
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(random_state=52)    

rf_clf.fit(x_train, y_train)
rf_pred = rf_clf.predict(x_test)
rf_pred_proba = rf_clf.predict_proba(x_test)
print("랜덤포레스트") 
get_clf_eval(y_test, rf_pred)
get_eval_by_threshold(y_test , rf_pred_proba[:,1].reshape(-1,1), thresholds)



In [ ]:
# 하이퍼파라미터 Grid-Search
from sklearn.model_selection import GridSearchCV

    # 탐색 범위 정의

grid_search = {'criterion': ['entropy', 'gini'],
            'max_depth': [2,4,6,8],
            'max_features': ['auto', 'sqrt'],
            'min_samples_leaf': [4,6,8,10],
            'min_samples_split': [4,6,8,10],
            'n_estimators': [20,40,60,80,100]}

model = RandomForestClassifier(random_state=52)
grid_search = GridSearchCV(estimator = model, param_grid = grid_search, 
                            cv = 5,verbose=0, n_jobs = -1)
grid_search.fit(x_train,y_train)

# 최선의 결과 출력
print('최적 하이퍼 파라미터 :{0}, 최적 평균 정확도 :{1:.3f}'.format(grid_search.best_params_, grid_search.best_score_))
a = grid_search.best_params_
keys=list(a.keys())
values = list(a.values())

rf_clf_best = RandomForestClassifier(random_state=52,criterion = values[0], max_depth = values[1],max_features=values[2], min_samples_leaf=values[3],  min_samples_split=values[4],n_estimators=values[5])
rf_clf_best.fit(x_train, y_train)
rf_best_pred = rf_clf_best.predict(x_test)
rf_best_pred_proba = rf_clf_best.predict_proba(x_test)   

print("랜덤포레스트 파라미터적용") 
get_clf_eval(y_test, rf_best_pred)
get_eval_by_threshold(y_test ,rf_best_pred_proba[:,1].reshape(-1,1), thresholds)

Lg

In [ ]:
# 1-4 로지스틱회귀
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression()

lr_clf.fit(x_train, y_train)
lr_pred = lr_clf.predict(x_test)
lr_pred_proba = lr_clf.predict_proba(x_test)
print("로지스틱회귀") 
get_clf_eval(y_test, lr_pred)
get_eval_by_threshold(y_test , lr_pred_proba[:,1].reshape(-1,1), thresholds)
    

In [ ]:
#로지스틱 회귀 최적 하이퍼 파라미터
from sklearn.model_selection import GridSearchCV


params = {'penalty': ['l2','l1','elasticnet','None'],
        'C':[0.01,0.1,1,3,5,10]}

grid_search = GridSearchCV(lr_clf, param_grid=params, scoring='accuracy', cv=5)
grid_search.fit(x_train,y_train)
print('최적 하이퍼 파라미어:{0}, 최적 평균 정확도:{1:.3f}'.format(grid_search.best_params_, grid_search.best_score_))

a = grid_search.best_params_
keys=list(a.keys())
values = list(a.values())

lr_clf_best = LogisticRegression(penalty= values[1], C=values[0])
lr_clf_best.fit(x_train, y_train)
lr_best_pred = lr_clf_best.predict(x_test)
lr_best_pred_proba = lr_clf_best.predict_proba(x_test)   

print("로지스틱회귀 파라미터적용") 
get_clf_eval(y_test, lr_best_pred)
get_eval_by_threshold(y_test ,lr_best_pred_proba[:,1].reshape(-1,1), thresholds)


xgb

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(random_state=17,
                    n_estimators=90,
                    max_depth=5,
                    learning_rate=0.03,
                    reg_lambda=2,
                    reg_alpha=0.02,
                    min_split_loss=2
                    )
xgb.fit(x_train, y_train)
xgb_pred = xgb.predict(x_test)
xgb_pred_proba = xgb.predict_proba(x_test)

print("xgboost 파라미터적용") 
get_clf_eval(y_test, xgb_pred)
get_eval_by_threshold(y_test ,xgb_pred_proba[:,1].reshape(-1,1), thresholds)